# Check a single event

Look into a single interesting event: Where did the missing hits go?

In [ ]:
import numpy as np
import pandas as pd
import uproot

from local_config import default_pos as pos
from plotting.hit_plots import plot_slices, plotly_3d


i_event = 622273
delta = 3
run_name = "Run_ILC_08042020_cosmic_it15_Ascii_build"
raw_path = f"data/raw/{run_name}.root"
b_cosmic = uproot.open(raw_path)
tree = b_cosmic["ecal"]
events = tree.arrays(entry_start=i_event - delta - 1, entry_stop=i_event + delta)

events[delta][[k for k in tree.keys() if not k.startswith("hit_")]].to_list()

In [ ]:
bcids = ["prev_bcid", "bcid", "next_bcid"]
df = pd.DataFrame([[event[bc] for bc in bcids] for event in events], columns=bcids)
df

In [ ]:
def get_dim(i, clean_up=True):
    def channel_remapping(dim):
        # There seems to be an issue in some layers.
        xy_mirrored = np.any([dim["z"] == mir for mir in [0, 2, 4, 9, 10]], axis=0)
        dim["x"] = dim["x"].copy()
        dim["y"] = dim["y"].copy()
        dim["x"][xy_mirrored] *= -1
        dim["y"][xy_mirrored] *= -1
        return dim

    ev = events[i]
    dim = {coord: ev["hit_" + coord] for coord in "xyz"}
    dim["e"] = ev["hit_energy"]
    if clean_up:
        mask = ev.hit_energy > 0
        mask = mask & (ev.hit_isHit == 1)
        for k, v in dim.items():
            dim[k] = v[mask]
    dim = {k: v.to_numpy() for k, v in dim.items()}
    dim = channel_remapping(dim)
    return dim


clean_up = False
for i in range(len(events)):
    dim = get_dim(i, clean_up)
    e_in_event = sum(abs(dim["e"]))
    print(f"{i}: {e_in_event:.1f}")
    if e_in_event > 50:
        plot_slices(pos, dim)
plotly_3d(get_dim(len(events) // 2, clean_up))

In [ ]:
import matplotlib.pyplot as plt


def bins(x):
    x_first = x[0] - (x[1] - x[0]) / 2
    x_last = x[-1] + (x[-1] - x[-2]) / 2
    x_mid = (x[1:] + x[:-1]) / 2
    return np.concatenate(([x_first], x_mid, [x_last]))


ev = events[len(events) // 2]
for k in ev.fields:
    if not k.startswith("hit_"):
        continue
    plt.figure()
    plt.title(k)
    plt.hist2d(
        ev.hit_x.to_numpy(),
        ev.hit_y.to_numpy(),
        weights=ev[k].to_numpy(),
        bins=(bins(pos["x"]), bins(pos["y"])),
        # norm=LogNorm(),
    )